In [1]:
import xlwings as xw
import time
import numpy as np  # Importamos numpy


# RUTA DEL LIBRO EXCEL Y LOS INDICES DE LAS HOJAS A PROCESAR
libro_path = r"D:\alan\workspace\isa\Calculo de Reserva y Flujos SCTR_Tasa Descuento.xlsx"
analisis_sheet_index = 4    
cru_sheet_index = 3
curva_descuento_sheet_index = 2
flujos_sheet_index = 5
n = 16561
chunk_size = 50  # Tamaño del chunk para procesar en bloques

# Rangos de datos
ranges = {
    "tipsin": f"N4:N{n}",
    "codparen": f"M4:M{n}",
    "fallecimiento": f"I4:I{n}",
    "siniestro": f"C4:C{n}",
    "fechacorte": f"D4:D{n}",
    "fechanacimiento": f"E4:E{n}",
    "sexo": f"F4:F{n}",
    "tceanual": f"H4:H{n}",
    "condsalu": f"G4:G{n}",
    "fechasiniestro": f"S4:S{n}",
    "ppenbas": f"Q4:Q{n}",
    "rempro": f"O4:O{n}",
    "inflacion": f"P4:P{n}",
    "reserva_output": f"T4:T{n}",
    "cobertura_output": f"R4:R{n}"
}

# Celdas de salida
celda_reserva_titular = "F24"
celda_reserva_beneficiario = "M24"
celda_reserva_sobrevivencia = "V24"


def procesar_chunk(data, cru, curva_descuento, start, end, suma_vector_flujos):
    # Obtenemos el subset de datos
    tipsin_chunk = data.range(f"N{start}:N{end}").value
    codparen_chunk = data.range(f"M{start}:M{end}").value
    fallecimiento_chunk = data.range(f"I{start}:I{end}").value
    siniestro_chunk = data.range(f"C{start}:C{end}").value
    fechacorte_chunk = data.range(f"D{start}:D{end}").value
    fechanacimiento_chunk = data.range(f"E{start}:E{end}").value
    sexo_chunk = data.range(f"F{start}:F{end}").value
    salud_chunk = data.range(f"G{start}:G{end}").value
    tce_chunk = data.range(f"H{start}:H{end}").value
    rempro_chunk = data.range(f"O{start}:O{end}").value
    infla_chunk = data.range(f"P{start}:P{end}").value
    ppenbas_chunk = data.range(f"Q{start}:Q{end}").value
    fechasiniestro_chunk = data.range(f"S{start}:S{end}").value
    cobertura_chunk = data.range(f"R{start}:R{end}").value
    reservas_chunk = [None] * (end - start + 1)
    vector_flujos = np.zeros(1332)
    
    # Procesamos cada fila del chunk
    for i in range(end - start + 1):
        index = start + i - 4
        print(f"Procesando fila {index}: SINIESTRO: {int(siniestro_chunk[i])}, TIPSIN: {tipsin_chunk[i]}")

        if tipsin_chunk[i] == "01":
            cru.range("T4").value = str(int(siniestro_chunk[i]))
            cru.range("T9").value = tce_chunk[i]
            curva_descuento.range("C3").value = cru.range("T10").value
            cru.range("T5").value = fechacorte_chunk[i]
            cru.range("T6").value = fechanacimiento_chunk[i]
            cru.range("T7").value = sexo_chunk[i]
            cru.range("T11").value = fallecimiento_chunk[i]
            cru.range("T12").value = salud_chunk[i]
            cru.range("T13").value = "'" + str(codparen_chunk[i])
            cru.range("T14").value = fechasiniestro_chunk[i]
            cru.range("T16").value = float(ppenbas_chunk[i]/100) if float(ppenbas_chunk[i]) > 1 else float(ppenbas_chunk[i])
            cru.range("T17").value = rempro_chunk[i]
            cru.range("T18").value = infla_chunk[i]

            reservas_chunk[i] = [cru.range(celda_reserva_sobrevivencia).value]
            vector_flujos = np.array(cru.range("X29:X1360").value)
            print(f"Reserva: {reservas_chunk[i]}")
        
        elif tipsin_chunk[i] == "02" and codparen_chunk[i] == '00':
            cru.range("D4").value = str(int(siniestro_chunk[i]))
            cru.range("D5").value = fechacorte_chunk[i]
            cru.range("D6").value = fechanacimiento_chunk[i]
            cru.range("D7").value = sexo_chunk[i]
            cru.range("D9").value = tce_chunk[i]
            curva_descuento.range("C3").value = cru.range("D10").value
            cru.range("D11").value = fallecimiento_chunk[i]
            cru.range("D12").value = salud_chunk[i]
            cru.range("D13").value = cobertura_chunk[i]
            cru.range("D14").value = "'" + str(codparen_chunk[i])
            cru.range("D16").value = float(ppenbas_chunk[i]/100) if float(ppenbas_chunk[i]) > 1 else float(ppenbas_chunk[i])
            cru.range("D17").value = rempro_chunk[i]
            cru.range("D18").value = infla_chunk[i]

            reservas_chunk[i] = [cru.range(celda_reserva_titular).value]
            vector_flujos = np.array(cru.range("G29:G1360").value)
            print(f"Reserva: {reservas_chunk[i]}")
            
        elif tipsin_chunk[i] == "02" and codparen_chunk[i] != '00':
            cru.range("K4").value = str(int(siniestro_chunk[i]))
            cru.range("K5").value = fechacorte_chunk[i]
            cru.range("K6").value = fechanacimiento_chunk[i]
            cru.range("K7").value = sexo_chunk[i]
            cru.range("K9").value = tce_chunk[i]
            curva_descuento.range("C3").value = cru.range("K10").value
            cru.range("K11").value = fallecimiento_chunk[i]
            cru.range("K12").value = salud_chunk[i]
            cru.range("K13").value = "'" + str(codparen_chunk[i])
            cru.range("K14").value = fechasiniestro_chunk[i]
            cru.range("K16").value = float(ppenbas_chunk[i]/100) if float(ppenbas_chunk[i]) > 1 else float(ppenbas_chunk[i])
            cru.range("K17").value = rempro_chunk[i]
            cru.range("K18").value = infla_chunk[i]

            reservas_chunk[i] = [cru.range(celda_reserva_beneficiario).value]
            vector_flujos = np.array(cru.range("P29:P1360").value)
            print(f"Reserva: {reservas_chunk[i]}")
            
        # Acumular cada componente de vector_flujos en suma_vector_flujos
        suma_vector_flujos += vector_flujos
    
    return reservas_chunk


def procesar_por_chunks(data, cru, curva_descuento, chunk_size):
    total_rows = n - 3
    reservas = []
    suma_vector_flujos = np.zeros(1332)

    for start in range(4, total_rows + 4, chunk_size):
        end = min(start + chunk_size - 1, total_rows + 3)
        reservas_chunk = procesar_chunk(data, cru, curva_descuento, start, end, suma_vector_flujos)
        reservas.extend(reservas_chunk)

    return reservas, suma_vector_flujos

# Iniciar temporizador
start_time = time.time()
libro = xw.Book(libro_path)
cru = libro.sheets[cru_sheet_index]
data = libro.sheets[analisis_sheet_index]
curva_descuento = libro.sheets[curva_descuento_sheet_index]
flujos = libro.sheets[flujos_sheet_index]

# Procesar en chunks
reservas, suma_vector_flujos = procesar_por_chunks(data, cru, curva_descuento, chunk_size)

# Guardar resultados
data.range(ranges["reserva_output"]).value = reservas
flujos.range("D8:D1339").value = suma_vector_flujos.reshape(-1, 1)

# Guardar y cerrar libro
libro.save()
libro.close()

# Tiempo total
end_time = time.time()
total_time = end_time - start_time
print(f"Proceso terminado en {total_time:.2f} segundos")

Procesando fila 0: SINIESTRO: 20100064, TIPSIN: 02
Reserva: [196943.31229345896]
Procesando fila 1: SINIESTRO: 20120029, TIPSIN: 02
Reserva: [256317.67936151612]
Procesando fila 2: SINIESTRO: 20120029, TIPSIN: 02
Reserva: [362923.5343378023]
Procesando fila 3: SINIESTRO: 20120098, TIPSIN: 02
Reserva: [170780.8500272081]
Procesando fila 4: SINIESTRO: 20120098, TIPSIN: 02
Reserva: [134086.61385059287]
Procesando fila 5: SINIESTRO: 20120098, TIPSIN: 02
Reserva: [380240.93518788606]
Procesando fila 6: SINIESTRO: 20120098, TIPSIN: 02
Reserva: [380240.93518788606]
Procesando fila 7: SINIESTRO: 20130005, TIPSIN: 02
Reserva: [52071.39867437804]
Procesando fila 8: SINIESTRO: 20130017, TIPSIN: 02
Reserva: [256896.71661716962]
Procesando fila 9: SINIESTRO: 20130017, TIPSIN: 02
Reserva: [41256.41553748226]
Procesando fila 10: SINIESTRO: 20130017, TIPSIN: 02
Reserva: [1414.9955632528647]
Procesando fila 11: SINIESTRO: 20130017, TIPSIN: 02
Reserva: [884.3266141103496]
Procesando fila 12: SINIESTRO: 